In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_formatting import get_effective_format
import pandas as pd

import pygsheets
import pandas as pd

import google.auth
from googleapiclient.discovery import build
import pygsheets
#from google.oauth2.credentials import Credentials
from google.oauth2.service_account import Credentials

x:\OTUS\.venv\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
x:\OTUS\.venv\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
x:\OTUS\.venv\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


#### consta

In [2]:
#SHEET_NAME = 'Copy of Спонсорские ТНТ 27.07.23 for script'

#CONST
KEY_JSON = 'google\\poised-backbone-191400-4e9fc454915f.json'

#SHEET_NAME = 'Спонсорские ТНТ 27.07.23'
SHEET_NAME = 'Copy of Спонсорские ТНТ 27.07.23 for script'import asyncio

import pandas as pd

from config import KEY_JSON, SHEET_INFO, TRIGGERS
from core.planner import GoogleSheetPlanner


MAIN_TABLE = 'ТАБЛИЧКА'
DESIGNER_TABLE = 'дизайнеры'


SHEET_NAMES = {
    'tasks': MAIN_TABLE,
    'designers': DESIGNER_TABLE,
}

SHEET_INFO = {
    'spreadsheet_name': SHEET_NAME,
    'sheet_names': SHEET_NAMES,
}

FIELD_MAP = {
    "brand": "БРЕНД",
    "format_": "ФОРМАТ",
    "project_name": "ПРОЕКТ",
    "customer": "ЗАКАЗЧИК",
    "designer": "ДИЗАЙНЕР",
    "timing": "Тайминг",
    "status": "Статус",
    "color": "color",
    "color_status": "color_status",
    "task_name": "task_name",
    "task_id": "task_id"
}

COLOR_STATUS = {
       '#FFFFFF': 'work',
       '#808080': 'wait',
       '#CCCCCC': 'wait',
       '#B6D7A8': 'done',
       '#D9D1E9': 'pre_done'    
}



In [ ]:


# Аутентификация
#creds_file = 'path_to_your_credentials.json'  # Замените на путь к вашему файлу учетных данных
gc = pygsheets.authorize(service_file=KEY_JSON )

# Открыть Google Таблицу по имени
sheet_name = SHEET_NAME  # Замените на имя вашей таблицы
sh = gc.open(sheet_name)

# Выберите лист по имени
worksheet = sh.worksheet_by_title(MAIN_TABLE)  # или другое имя листа

# Получить все значения из листа
values = worksheet.get_all_values()

df = pd.DataFrame(values[1:], columns=values[0])

# Удалить пустые строки
df = df[df.astype(bool).any(axis=1)]

# Получить цвета ячеек
colors = []

l = len(df) + 1

cell_colors = worksheet.get_col(1, returnas='cell')  # Получаем список объектов Cell для первой колонки
for i, cell in enumerate(cell_colors):
    if i >= l:
        break
    color = color = [c if c else 1 for c in cell.color]
    # Преобразовать цвет в строковый формат, например "#RRGGBB"
    color_str = "#{:02X}{:02X}{:02X}".format(int(color[0]*255), int(color[1]*255), int(color[2]*255))
    colors.append(color_str)

df['Color'] = colors[1:len(df)+1]  # учитываем только необходимое количество цветов
df['Row Number'] = range(1, len(df) + 1)



df


In [ ]:
color_legend = {
    '#FFFFFF': 'work',
    '#808080': 'wait',
    '#CCCCCC': 'wait',
    '#B6D7A8': 'done',
    '#D9D1E9': 'pre_done'    
}

df['Status'] = df['Color'].apply(lambda x: color_legend[x] if x in color_legend else 'unknown')

df

In [ ]:
df['name'] = df.apply(lambda row: row['БРЕНД'] + " " + row['ПРОЕКТ'] + " " + row['ФОРМАТ'].split('\n')[0] + ' ' + str(row['Row Number']), axis=1)
df

In [ ]:
filtered_df = df[df['Status'] == "work"]

# Группировка по дизайнеру и агрегация проектов в список
designer_projects = filtered_df.groupby('ДИЗАЙНЕР')['name'].apply(list).to_dict()

designer_projects

In [128]:
df.columns

Index(['БРЕНД', 'ФОРМАТ', 'ПРОЕКТ', 'ЗАКАЗЧИК', 'ДИЗАЙНЕР', 'Тайминг',
       'Статус ', 'Color', 'Row Number', 'Status', 'name'],
      dtype='object')

In [133]:
import google.auth
from googleapiclient.discovery import build
import pygsheets
#from google.oauth2.credentials import Credentials
from google.oauth2.service_account import Credentials

class BatchNotesUpdater:
    def __init__(self, spreadsheet_id):
        self.spreadsheet_id = spreadsheet_id
        self.requests = []
        #credentials, _ = google.auth.default()
        #credentials = Credentials.from_authorized_user(KEY_JSON)
        credentials = Credentials.from_service_account_file(KEY_JSON)
        self.service = build('sheets', 'v4', credentials=credentials)

    def add_note(self, sheet_id, row, col, note):
        """Добавить заметку к ячейке."""
        request = {
            "updateCells": {
                "rows": {
                    "values": {
                        "note": note
                    }
                },
                "fields": "note",
                "range": {
                    "sheetId": sheet_id,
                    "startRowIndex": row - 1,
                    "endRowIndex": row,
                    "startColumnIndex": col - 1,
                    "endColumnIndex": col
                }
            }
        }
        self.requests.append(request)

    def add_value(self, sheet_id, row, col, value):
        """Добавить значение в ячейку."""
        request = {
            "updateCells": {
                "rows": {
                    "values": {
                        "userEnteredValue": {
                            "stringValue": value
                        }
                    }
                },
                "fields": "userEnteredValue",
                "range": {
                    "sheetId": sheet_id,
                    "startRowIndex": row - 1,
                    "endRowIndex": row,
                    "startColumnIndex": col - 1,
                    "endColumnIndex": col
                }
            }
        }
        self.requests.append(request)

    def execute(self):
        """Выполнить пакетное обновление."""
        if self.requests:
            batch_update_request = self.service.spreadsheets().batchUpdate(spreadsheetId=self.spreadsheet_id, body={"requests": self.requests})
            batch_update_request.execute()
            self.requests = []





# Инициализация
gc = pygsheets.authorize(service_file=KEY_JSON)
sh = gc.open(SHEET_NAME)
designers_worksheet = sh.worksheet_by_title(DESIGNER_TABLE)

# Очистить лист
designers_worksheet.clear()

# Отсортировать имена дизайнеров по алфавиту
designer_names = sorted(designer_projects.keys())

# Запись имен дизайнеров в первую строку
designers_worksheet.update_values('B2', [designer_names])

# Инициализация объекта BatchNotesUpdater
updater = BatchNotesUpdater(sh.id)

# Предполагаем, что у вас максимум 100 строк и 10 столбцов, где могут быть заметки.
# Вы можете изменить эти значения на более подходящие для вашего случая.
for row in range(1, 101):
    for col in range(1, 11):
        updater.add_note(designers_worksheet.id, row, col, "")
updater.execute()

# Обновление ячеек с проектами и добавление заметок
for col, designer in enumerate(designer_names, 2):
    projects = designer_projects[designer]
    for row, project in enumerate(projects, 3):  # начинаем с 2-й строки
        # Обновление значения ячейки
        #designers_worksheet.update_value((row, col), project)
        updater.add_value(designers_worksheet.id, row, col, project)
        
        # Получите значение из столбца "Тайминг" для данного проекта
        timing_value = filtered_df[filtered_df['name'] == project]['Тайминг'].values[0]
        
        # Добавление заметки
        updater.add_note(designers_worksheet.id, row, col, timing_value)
        #updater.add_note(designers_worksheet.id, row, col, '')

# Выполнение пакетного обновления
updater.execute()

In [251]:
class GoogleSheetInfo:
    def __init__(self, spreadsheet_name, tasks_sheet_name, designers_sheet_name):
        self.spreadsheet_name = spreadsheet_name
        self.tasks_sheet_name = tasks_sheet_name
        self.designers_sheet_name = designers_sheet_name



### class

In [97]:
a = []
class GoogleSheetInfo:
    def __init__(self, spreadsheet_name, sheet_names):
        """
        Args:
        - spreadsheet_name (str): Название таблицы.
        - sheet_names (dict): Словарь с именами листов. Например, {"tasks": "ТАБЛИЧКА", "designers": "дизайнеры"}
        """
        self.spreadsheet_name = spreadsheet_name
        self.sheets = sheet_names

    def get_sheet_name(self, key):
        """Получить имя листа по ключу."""
        return self.sheets.get(key)

#1. Доменный слой
#Создадим основные сущности:

class Task:
    def __init__(self, brand, format_, project_name, customer, designer, timing, status, color, color_status, task_name, task_id):
        self.brand = brand
        self.format_ = format_
        self.project_name = project_name
        self.customer = customer
        self.designer = designer
        self.timing = timing
        self.status = status
        self.color = color
        self.color_status = color_status
        self.task_name = task_name
        self.task_id = task_id
    
    def __repr__(self):
        return self.task_name


class Designer:
    def __init__(self, name):
        self.name = name

#2. Репозитории
#Определим интерфейсы и базовую реализацию:

from abc import ABC, abstractmethod
from typing import List

class TaskRepository(ABC):
    ''' Репозиторий для работы с задачами'''
    @abstractmethod
    def get_all_tasks(self) -> List[Task]:
        pass

class GoogleSheetsService:
    ''' Сервис для работы с Google Таблицами'''
    def __init__(self, credentials_path):             
        credentials = Credentials.from_service_account_file(credentials_path)
        self.sheets_service = build('sheets', 'v4', credentials=credentials)
        self.drive_service = build('drive', 'v3', credentials=credentials)
        self.requests = []
        self.sheet_id_cache = {}
        self.get_spreadsheet_id_cache = {}

    def get_spreadsheet_id_by_name(self, spreadsheet_name):
        """Получение ID таблицы по ее имени."""        
        if spreadsheet_name in self.get_spreadsheet_id_cache:
            return self.get_spreadsheet_id_cache[spreadsheet_name]
        results = self.drive_service.files().list(q=f"name='{spreadsheet_name}'", fields="files(id, name)").execute()
        files = results.get('files', [])
        
        if not files:
            raise ValueError(f"Spreadsheet '{spreadsheet_name}' not found.")
        
        spreadsheet_id = files[0]['id']
        self.get_spreadsheet_id_cache[spreadsheet_name] = spreadsheet_id

        return spreadsheet_id

    def get_dataframe(self, spreadsheet_name, worksheet_name):
        '''Получить DataFrame по имени таблицы и имени листа.'''
        spreadsheet_id = self.get_spreadsheet_id_by_name(spreadsheet_name)
        result = self.sheets_service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id, 
            range=worksheet_name
        ).execute()
        values = result.get('values', [])
        df = pd.DataFrame(values[1:], columns=values[0])  # предполагая, что первая строка содержит заголовки
        return df
    
    def get_cell_colors(self, spreadsheet_name, worksheet_name, range_='A1:Z1000'):
        ''' Получить цвета ячеек листа'''        
        spreadsheet_id = self.get_spreadsheet_id_by_name(spreadsheet_name)
        
        result = self.sheets_service.spreadsheets().get(
            spreadsheetId=spreadsheet_id, 
            fields="sheets(data.rowData.values.userEnteredFormat.backgroundColor,properties.title)",
        ).execute()
        
        sheet = next(item for item in result['sheets'] if item["properties"]["title"] == worksheet_name)
        rows = sheet['data'][0].get('rowData', [])
    
        start_col, start_row, end_col, end_row = self.parse_range(range_)
    
        colors = []
        for r, row in enumerate(rows[start_row:end_row+1]):
            for c, cell in enumerate(row['values'][start_col:end_col+1]):
                color = cell['userEnteredFormat']['backgroundColor']
                colors.append(self.color_to_str(color))
    
        return colors
    
    def parse_range(self, range_):
        ''' Конвертировать диапазон A1:Z1000 в индексы (0, 0, 25, 1000) '''
        col_row = range_.split(':')
        start_col, start_row = self.cell_to_indices(col_row[0])
        end_col, end_row = self.cell_to_indices(col_row[1])
        return start_col, start_row, end_col, end_row
    
    def cell_to_indices(self, cell):
        ''' Конвертировать A1 в (0, 0) '''
        col = ord(cell[0]) - 65
        row = int(cell[1:]) - 1
        return col, row
    
    def color_to_str(self, color):
        ''' Конвертировать словарь цвета в строку формата #RRGGBB '''
        return "#{:02X}{:02X}{:02X}".format(int(color['red']*255), int(color['green']*255), int(color['blue']*255))
 
    def update_cell(self, spreadsheet_name: str, sheet_name: str, row: int, col: int, **kwargs):
        ''' Добавить запрос на обновление ячейки в пакет '''
        fields = []
        cell_data = {}
        
        if "value" in kwargs:
            value = kwargs["value"]
            if pd.isna(value):
                value = ""
            cell_data["userEnteredValue"] = {"stringValue": value}
            fields.append("userEnteredValue")
        
        if "note" in kwargs:
            value = kwargs["note"]
            if pd.isna(value):
                value = ""
            cell_data["note"] = value
            fields.append("note")
        
        if "color" in kwargs:
            rgb = kwargs["color"]
            cell_data["userEnteredFormat"] = {"backgroundColor": {"red": rgb[0], "green": rgb[1], "blue": rgb[2]}}
            fields.append("userEnteredFormat.backgroundColor")

        #print(cell_data)
        
        sheet_id = self.get_sheet_id_by_name(spreadsheet_name, sheet_name)
        
        request = [spreadsheet_name,{
            "updateCells": {
                "rows": {
                    "values": [cell_data]
                },
                "fields": ",".join(fields),
                "range": {
                    "sheetId": sheet_id,
                    "startRowIndex": row - 1,
                    "endRowIndex": row,
                    "startColumnIndex": col - 1,
                    "endColumnIndex": col
                }
            }
        }]
        self.requests.append(request)
        #self.execute_updates(spreadsheet_name)


    def execute_updates(self, spreadsheet_name: str, requests: list=None):
        ''' Выполнить пакетное обновление '''
        if requests:
            req = requests
        else:
            req = [req[1] for req in self.requests if req[0] == spreadsheet_name]
        
        #relevant_requests = [req["updateCells"] for req in self.requests if req["spreadsheet_name"] == spreadsheet_name]
        spreadsheet_id = self.get_spreadsheet_id_by_name(spreadsheet_name)
        batch_update_request = self.sheets_service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body={"requests": req})
        batch_update_request.execute()
        #
        #if relevant_requests:
        #    batch_update_request = self.sheets_service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body={"requests": relevant_requests})
        #    batch_update_request.execute()
        #    self.requests = [req for req in self.requests if req["spreadsheet_name"] != spreadsheet_name]
        if not requests:
            self.requests = [req for req in self.requests if req[0] != spreadsheet_name]

    def clear_requests(self):
        ''' Очистить пакет обновлений '''
        self.requests = []

    def clear_cells(self, spreadsheet_name, sheet_name, range_='A1:Z1000'):
        ''' Очистить заданный диапазон ячеек от значений и заметок '''        
        spreadsheet_id = self.get_spreadsheet_id_by_name(spreadsheet_name)
        sheet_id = self.get_sheet_id_by_name(spreadsheet_name, sheet_name)
        
        # Переводим диапазон в индексы
        start_col, start_row, end_col, end_row = self.parse_range(range_)
        
        request = {
            "updateCells": {
                "fields": "userEnteredValue,note",
                "range": {
                    "sheetId": sheet_id,
                    "startRowIndex": start_row,
                    "endRowIndex": end_row + 1,
                    "startColumnIndex": start_col,
                    "endColumnIndex": end_col + 1
                }
            }
        }
        
        self.execute_updates(spreadsheet_name, [request])


    def get_sheet_id_by_name(self, spreadsheet_name, worksheet_name):
        ''' Получить ID листа по имени '''
        cache_key = f"{spreadsheet_name}_{worksheet_name}"
        if cache_key in self.sheet_id_cache:
            return self.sheet_id_cache[cache_key]

        spreadsheet_id = self.get_spreadsheet_id_by_name(spreadsheet_name)
        result = self.sheets_service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
        for sheet in result['sheets']:
            if sheet["properties"]["title"] == worksheet_name:
                sheet_id = sheet["properties"]["sheetId"]
                self.sheet_id_cache[cache_key] = sheet_id
                return sheet_id


class GoogleSheetsTaskRepository(TaskRepository):
    ''' Репозиторий для работы с задачами в Google Таблицах'''
    def __init__(self, sheet_info: GoogleSheetInfo, service: GoogleSheetsService):
        self.sheet_info = sheet_info
        self.service = service    
        self.df = None 

    def _load_and_process_data(self):
        ''' Загрузить и обработать данные из Google Таблицы'''
        df = self.service.get_dataframe(self.sheet_info.spreadsheet_name, self.sheet_info.get_sheet_name("tasks"))
        color_range = f'A1:A{len(df)}'
        colors = self.service.get_cell_colors(self.sheet_info.spreadsheet_name, self.sheet_info.get_sheet_name("tasks"), color_range)
        df['color'] = colors
        df['color_status'] = df['color'].apply(self._determine_status_from_color)
        df['task_id'] = df.index + 1
        df['task_name'] = df['БРЕНД'] + " " + df['ПРОЕКТ'] + " " + df['ФОРМАТ'].str.split().str[0]
        self.df = df

    def _check_df(self):
        ''' Проверить наличие данных'''
        if self.df is None:
            self._load_and_process_data()

    def _df_to_task(self, df):
        ''' Преобразовать DataFrame в список задач'''
        projects_list = []
        for idx, row in df.iterrows():
            project = {key: row[value] for key, value in FIELD_MAP.items()}
            project = Task(**project)
            projects_list.append(project)
        return projects_list
    
    def _filter(self, column_name, value):
        ''' Отфильтровать DataFrame по значению в колонке'''
        self._check_df()
        return self.df[self.df[column_name] == value]

    def get_all_tasks(self) -> List[Task]:
        ''' Получить все задачи'''
        self._check_df()
        projects = self._df_to_task(self.df)
        return projects
    
    def _determine_status_from_color(self, color):        
        ''' Определить статус по цвету'''
        cs = COLOR_STATUS
        return cs.get(color, "unknown")
    
    def get_task_by(self, column_name, value):
        ''' Получить задачи по значению в колонке'''
        df = self._filter(column_name, value)
        return self._df_to_task(df)

    def get_task_by_id(self, project_id):
        return self.get_task_by('task_id', project_id)[0]

    def get_task_by_name(self, project_name):
        return self.get_task_by('task_name', project_name)[0]

    def get_task_by_des(self, designer_name):
        return self.get_task_by('ДИЗАЙНЕР', designer_name)

    def get_task_by_cstatus(self, color_status):
        ''' Получить задачи по цветовому статусу'''
        return self.get_task_by('color_status', color_status)



In [100]:
#3. Use Cases
#Реализуем основную логику:

class GoogleSheetsTaskManager:
    def __init__(self, task_repository):
        self.repository = task_repository
        self.tasks = None
        self.designers = None

    def task_to_table(self, cstatus=['work']):
        ''' Получить задачи в виде таблицы'''
        self.update()
        tasks = [task for task in self.tasks if task.color_status in cstatus]
        # Получаем ID дизайнерского листа
        spreadsheet_name = self.repository.sheet_info.spreadsheet_name
        sheet_name = self.repository.sheet_info.get_sheet_name("designers")
        #sheet_id = self.repository.service.get_sheet_id_by_name(spreadsheet_name, designers_sheet_name)
        # Очищаем лист
        self.repository.service.clear_cells(spreadsheet_name, sheet_name)

        # Устанавливаем начальные координаты
        start_row, col = 2, 2
        row = start_row + 1

        # Сортируем дизайнеров по имени
        for designer in sorted(self.designers):
            tasks_for_designer = [task for task in tasks if task.designer == designer]

            # Если у дизайнера нет задач, переходим к следующему
            if not tasks_for_designer:
                continue

            # Добавляем имя дизайнера
            self.repository.service.update_cell(
                spreadsheet_name, sheet_name, start_row, col, value=designer
            )

            for task in tasks_for_designer:
                self.repository.service.update_cell(
                    spreadsheet_name, sheet_name, row, col, 
                    value=task.task_name, note=task.timing
                )
                row += 1

            col += 1
            row = start_row + 1

        #print(row, col)
        self.repository.service.execute_updates(spreadsheet_name)

    def update(self):
        ''' Обновить данные из репозитория'''
        self.tasks = self.repository.get_all_tasks()
        self.designers = set(task.designer for task in self.tasks if task.designer)


In [101]:
sheet_info = GoogleSheetInfo(**SHEET_INFO)
sheets_service = GoogleSheetsService(KEY_JSON)
projects = GoogleSheetsTaskRepository(sheet_info, sheets_service)
task_manager = GoogleSheetsTaskManager(projects)
task_manager.task_to_table(['work', 'pre_done'])